In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

spark= SparkSession.builder.appName('task_1').getOrCreate()

# data = [(10,),
#         (20,),
#         (20,),
#         (30,),
#         (30,),
#         (30,),
#         (40,),
#         (40,),
#         (40,),
#         (40,),
#         ]
# df= spark.createDataFrame(data,['MARKS'])

data=[
    ('A',1,'5/23/2003',100),
    ('B',2,'5/23/2003',200),
    ('A',4,'5/23/2003',200),
    ('B',3,'5/22/2003',200),
    ('A',5,'5/21/2003',500)
]
df= spark.createDataFrame(data,['GRADE','ROll_NO','DATE','MARKS'])
df = df.withColumn("Date", to_date("DATE", "M/d/yyyy"))
# sorted_df2 = df.orderBy("Date")
display(df)

GRADE,ROll_NO,Date,MARKS
A,1,2003-05-23,100
B,2,2003-05-23,200
A,4,2003-05-23,200
B,3,2003-05-22,200
A,5,2003-05-21,500


In [0]:

def transform_dataframe(input_df):

    dfs = []
    for i in input_df.columns:
        id_count = input_df.groupBy(i).agg(count('*').alias('count'))
        id_count_sorted = id_count.orderBy(['count', i], ascending=[False, True])
        id_count_sorted.show()
        percentage = format_number((id_count_sorted['count'] / input_df.count()) * 100, 1)
        print(percentage)
        percentage_with_percent = concat(i,lit(' - '),percentage, lit('%'))
        print(percentage_with_percent)
        value_count_df = id_count_sorted.withColumn(i, percentage_with_percent)
        value_count_df.show()
        value_count_df = value_count_df.select(i)
        value_count_df.show()
        value_count_df = value_count_df.withColumn("ID", monotonically_increasing_id())
        value_count_df.show()
        dfs.append(value_count_df)
        print('-----------------------------')
        x = dfs[0]
        for i in dfs[1:]:
            x = x.join(i, "ID", "full")

    #print(type(x))
    return x.drop("ID")

transformed_df = transform_dataframe(df)

display(transformed_df)


+-----+-----+
|GRADE|count|
+-----+-----+
|    A|    3|
|    B|    2|
+-----+-----+

Column<'format_number(((count / 5) * 100), 1)'>
Column<'concat(GRADE,  - , format_number(((count / 5) * 100), 1), %)'>
+---------+-----+
|    GRADE|count|
+---------+-----+
|A - 60.0%|    3|
|B - 40.0%|    2|
+---------+-----+

+---------+
|    GRADE|
+---------+
|A - 60.0%|
|B - 40.0%|
+---------+

+---------+---+
|    GRADE| ID|
+---------+---+
|A - 60.0%|  0|
|B - 40.0%|  1|
+---------+---+

-----------------------------
+-------+-----+
|ROll_NO|count|
+-------+-----+
|      1|    1|
|      2|    1|
|      3|    1|
|      4|    1|
|      5|    1|
+-------+-----+

Column<'format_number(((count / 5) * 100), 1)'>
Column<'concat(ROll_NO,  - , format_number(((count / 5) * 100), 1), %)'>
+---------+-----+
|  ROll_NO|count|
+---------+-----+
|1 - 20.0%|    1|
|2 - 20.0%|    1|
|3 - 20.0%|    1|
|4 - 20.0%|    1|
|5 - 20.0%|    1|
+---------+-----+

+---------+
|  ROll_NO|
+---------+
|1 - 20.0%|
|2 - 20.0%

GRADE,ROll_NO,Date,MARKS
A - 60.0%,1 - 20.0%,2003-05-23 - 60.0%,200 - 60.0%
B - 40.0%,2 - 20.0%,2003-05-21 - 20.0%,100 - 20.0%
null,3 - 20.0%,2003-05-22 - 20.0%,500 - 20.0%
null,4 - 20.0%,null,null
null,5 - 20.0%,null,null


In [0]:
def find_top_10(numbers):

    # Sort the numbers in descending order.
    numbers.sort(reverse=True)

    # Count the occurrences of each number
    occurrences = {}
    for num in numbers:
        occurrences[num] = occurrences.get(num, 0) + 1

    top_10 = []
    seen = set()
    for num in numbers:
        if len(top_10) >= 1:
            break
        if num not in seen:
            top_10.append(num)
            seen.add(num)

    result = []
    for num in top_10:
        result.extend([num] * occurrences[num])

    return result

numbers = [115, 113, 2, 111, 4, 117, 6, 8, 10, 9, 15, 20, 20,120]

# Extend the list with additional numbers
additional_numbers = list(range(21, 121))  # Adding numbers from 21 to 120
numbers.extend(additional_numbers)
numbers2=[60,40]
numbers3=[20,20,20,20,20]
# Find the top 10 values
top_10 = find_top_10(numbers3)

print(top_10)


[20, 20, 20, 20, 20]
